<a href="https://colab.research.google.com/github/DIWEERAPURA/Nature-Inspired-Computing-Algorithms---Real-world-Usage/blob/main/ACO_Exam_Time_Tables-Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Google Colab–ready Advanced ACO for Exam Timetabling

import numpy as np
import random

# ---------------------------
# Setup and Parameterization
# ---------------------------

# For reproducibility in Colab:
np.random.seed(42)
random.seed(42)

# Problem parameters
NUM_EXAMS = 10
NUM_TIMESLOTS = 5
NUM_ROOMS = 3
ROOM_CAPACITY = [30, 50, 40]         # Capacities for each room
NUM_PROFESSORS = NUM_EXAMS           # One professor per exam

# ACO parameters
NUM_ANTS = 20
NUM_ITERATIONS = 100
PHEROMONE_INIT = 1.0
EVAPORATION_RATE = 0.5
ALPHA = 1.0                        # Importance of pheromone
BETA = 2.0                         # Importance of heuristic info
ELITISM_FACTOR = 2.0               # Extra reinforcement for global best

# ---------------------------
# Input Data and Precomputation
# ---------------------------

# Random binary matrix: 50 students and their exam enrollments (0: not taking, 1: taking)
STUDENT_EXAM_MATRIX = np.random.randint(0, 2, (50, NUM_EXAMS))

# Random binary matrix for professor availability: each professor (per exam) over available timeslots
PROFESSOR_AVAILABILITY = np.random.randint(0, 2, (NUM_PROFESSORS, NUM_TIMESLOTS))

# Dictionary specifying preferred timeslots for some exams/professors.
# (If an exam’s timeslot is preferred, the heuristic gets a bonus.)
PREFERRED_TIMES = {0: [0, 1], 1: [2], 2: [1, 3]}

# Room adjacency matrix (can be used for further extensions)
ROOM_ADJACENCY = np.random.rand(NUM_ROOMS, NUM_ROOMS)

# Precompute the conflict matrix:
# conflict_matrix[i, j] = number of students taking both exam i and exam j.
conflict_matrix = np.dot(STUDENT_EXAM_MATRIX.T, STUDENT_EXAM_MATRIX)
np.fill_diagonal(conflict_matrix, 0)
# Compute per-exam conflict penalty (sum over conflicts with other exams)
conflict_penalties = np.sum(conflict_matrix, axis=1)

# Initialize pheromone matrix with dimensions: (exams x timeslots x rooms)
pheromone = np.ones((NUM_EXAMS, NUM_TIMESLOTS, NUM_ROOMS)) * PHEROMONE_INIT

# ---------------------------
# Heuristic Function
# ---------------------------

def calculate_heuristic_matrix():
    """
    Compute a heuristic value for each (exam, timeslot, room) based on:
      - Professor availability (binary: 0 or 1)
      - Preferred timeslot bonus (1 if preferred, else 0.5)
      - Room capacity (higher capacity is favorable)
      - Precomputed conflict penalty (more conflicts lower the heuristic)
    """
    heuristic = np.zeros((NUM_EXAMS, NUM_TIMESLOTS, NUM_ROOMS))
    for exam in range(NUM_EXAMS):
        for timeslot in range(NUM_TIMESLOTS):
            availability = PROFESSOR_AVAILABILITY[exam, timeslot]
            # Apply bonus if the timeslot is preferred for this exam; else, a lower base value.
            pref_bonus = 1.0 if timeslot in PREFERRED_TIMES.get(exam, []) else 0.5
            for room in range(NUM_ROOMS):
                capacity = ROOM_CAPACITY[room]
                # The heuristic favors higher availability, preferred times, and larger capacity,
                # while penalizing exams with many conflicts.
                heuristic[exam, timeslot, room] = (availability * pref_bonus * capacity) / (1 + conflict_penalties[exam])
    return heuristic

# ---------------------------
# Fitness Evaluation
# ---------------------------

def evaluate_solution(solution):
    """
    Evaluate the quality of a timetabling solution.
    Penalties include:
      - Student exam overlaps (if a student is scheduled for multiple exams in the same timeslot)
      - Assignments where the professor is unavailable
      - Non-preferred timeslot assignment
    Lower penalty values indicate better solutions.
    """
    penalty = 0

    # Student overlap penalty: count duplicate timeslot assignments for each student.
    for student in range(STUDENT_EXAM_MATRIX.shape[0]):
        # Get exams that the student is taking.
        exam_indices = np.where(STUDENT_EXAM_MATRIX[student] == 1)[0]
        # Retrieve assigned timeslots for those exams.
        assigned_times = [solution[exam][0] for exam in exam_indices]
        # Each duplicate in timeslot assignment contributes to the penalty.
        penalty += len(assigned_times) - len(set(assigned_times))

    # Penalty for professor unavailability and non-preferred timeslot.
    for exam, (timeslot, room) in solution.items():
        if not PROFESSOR_AVAILABILITY[exam, timeslot]:
            penalty += 10   # Heavy penalty for scheduling when professor is unavailable
        if timeslot not in PREFERRED_TIMES.get(exam, []):
            penalty += 2    # Penalty for non-preferred timeslot

    return penalty

# ---------------------------
# Advanced ACO Algorithm
# ---------------------------

def aco_timetabling():
    global pheromone
    best_solution = None
    best_fitness = float("inf")

    for iteration in range(NUM_ITERATIONS):
        solutions = []
        fitness_scores = []

        # Compute heuristic matrix once per iteration (it remains static in this model)
        heuristic = calculate_heuristic_matrix()

        # Each ant constructs a complete solution.
        for ant in range(NUM_ANTS):
            solution = {}
            for exam in range(NUM_EXAMS):
                # Build the probability matrix for each (timeslot, room) combination.
                prob_matrix = np.zeros((NUM_TIMESLOTS, NUM_ROOMS))
                for timeslot in range(NUM_TIMESLOTS):
                    for room in range(NUM_ROOMS):
                        prob_matrix[timeslot, room] = (pheromone[exam, timeslot, room] ** ALPHA) * (heuristic[exam, timeslot, room] ** BETA)

                # Normalize the probability distribution (with a safeguard for zero sum)
                total = np.sum(prob_matrix)
                if total == 0:
                    probabilities = np.full(prob_matrix.shape, 1/(NUM_TIMESLOTS * NUM_ROOMS))
                else:
                    probabilities = prob_matrix / total

                # Choose one timeslot-room combination based on the computed probabilities.
                flat_index = np.random.choice(np.arange(NUM_TIMESLOTS * NUM_ROOMS), p=probabilities.flatten())
                chosen_timeslot, chosen_room = np.unravel_index(flat_index, (NUM_TIMESLOTS, NUM_ROOMS))
                solution[exam] = (chosen_timeslot, chosen_room)

            fitness = evaluate_solution(solution)
            solutions.append(solution)
            fitness_scores.append(fitness)

        # ---------------------------
        # Pheromone Update
        # ---------------------------

        # Evaporation step reduces all pheromone levels.
        pheromone *= (1 - EVAPORATION_RATE)

        # Deposit pheromone based on each ant's solution quality.
        for sol, fit in zip(solutions, fitness_scores):
            for exam, (timeslot, room) in sol.items():
                pheromone[exam, timeslot, room] += 1.0 / (fit + 1e-6)

        # Identify the best solution in the current iteration.
        current_best_fit = min(fitness_scores)
        if current_best_fit < best_fitness:
            best_fitness = current_best_fit
            best_solution = solutions[fitness_scores.index(current_best_fit)]

        # Elitist update: extra reinforcement for the globally best solution.
        for exam, (timeslot, room) in best_solution.items():
            pheromone[exam, timeslot, room] += ELITISM_FACTOR / (best_fitness + 1e-6)

        # Display progress.
        print(f"Iteration {iteration + 1}/{NUM_ITERATIONS}: Best Fitness = {best_fitness}")

    return best_solution, best_fitness

# ---------------------------
# Run the Algorithm and Display Results
# ---------------------------

# In a Colab cell, simply run the code to see the results.
best_solution, best_fitness = aco_timetabling()
print("\n=== Final Best Timetabling Solution ===")
for exam in range(NUM_EXAMS):
    timeslot, room = best_solution[exam]
    print(f"Exam {exam}: Timeslot {timeslot}, Room {room}")
print("Best Fitness:", best_fitness)


Iteration 1/100: Best Fitness = 86
Iteration 2/100: Best Fitness = 85
Iteration 3/100: Best Fitness = 85
Iteration 4/100: Best Fitness = 85
Iteration 5/100: Best Fitness = 85
Iteration 6/100: Best Fitness = 85
Iteration 7/100: Best Fitness = 85
Iteration 8/100: Best Fitness = 85
Iteration 9/100: Best Fitness = 85
Iteration 10/100: Best Fitness = 80
Iteration 11/100: Best Fitness = 79
Iteration 12/100: Best Fitness = 79
Iteration 13/100: Best Fitness = 79
Iteration 14/100: Best Fitness = 79
Iteration 15/100: Best Fitness = 79
Iteration 16/100: Best Fitness = 79
Iteration 17/100: Best Fitness = 79
Iteration 18/100: Best Fitness = 79
Iteration 19/100: Best Fitness = 79
Iteration 20/100: Best Fitness = 79
Iteration 21/100: Best Fitness = 79
Iteration 22/100: Best Fitness = 79
Iteration 23/100: Best Fitness = 79
Iteration 24/100: Best Fitness = 79
Iteration 25/100: Best Fitness = 79
Iteration 26/100: Best Fitness = 79
Iteration 27/100: Best Fitness = 79
Iteration 28/100: Best Fitness = 79
I

In [4]:
# Advanced ACO for Exam Timetabling with Room and Examiner Assignment
# Google Colab–ready cell

import numpy as np
import random

# ---------------------------
# Reproducibility
# ---------------------------
np.random.seed(42)
random.seed(42)

# ---------------------------
# Problem Data
# ---------------------------

# Exams, timeslots, exam halls, and examiners.
exams = ['A','B','C','D','E','F','G','H','I','J']
timeslot_labels = ['9:00 AM to 11:00 AM', '12:00 PM to 2:00 PM']
exam_halls = ['Exam Hall 1', 'Exam Hall 2', 'Exam Hall 3']
examiners = ['Examiner 1', 'Examiner 2', 'Examiner 3', 'Examiner 4', 'Examiner 5']

num_exams = len(exams)
num_timeslots = len(timeslot_labels)
num_rooms = len(exam_halls)
num_examiners = len(examiners)
num_students = 100  # Number of students

# Define room capacities corresponding to exam halls.
ROOM_CAPACITY = [30, 50, 40]

# ---------------------------
# Student and Professor Data
# ---------------------------

# Generate a random binary matrix (students x exams) indicating student enrollment.
STUDENT_EXAM_MATRIX = np.random.randint(0, 2, (num_students, num_exams))

# For simplicity, assume each exam is supervised by a dedicated professor.
# Create a binary availability matrix for each exam's professor across timeslots.
PROFESSOR_AVAILABILITY = np.random.randint(0, 2, (num_exams, num_timeslots))

# Define preferred times for each exam (if available).
# For instance, exam 0 prefers timeslot 0, exam 1 prefers timeslot 1, etc.
PREFERRED_TIMES = {
    0: [0],
    1: [1],
    2: [0],
    3: [1],
    4: [0],
    # Others can be omitted (no strong preference) or set as needed.
}

# Precompute a conflict matrix: conflict_matrix[i,j] counts number of students taking both exam i and exam j.
conflict_matrix = np.dot(STUDENT_EXAM_MATRIX.T, STUDENT_EXAM_MATRIX)
np.fill_diagonal(conflict_matrix, 0)
# For each exam, the conflict penalty is the total conflicts with other exams.
conflict_penalties = np.sum(conflict_matrix, axis=1)

# ---------------------------
# ACO Parameters
# ---------------------------
NUM_ANTS = 10
NUM_ITERATIONS = 100
PHEROMONE_INIT = 1.0
EVAPORATION_RATE = 0.5
ALPHA = 1.0      # Pheromone importance
BETA = 2.0       # Heuristic importance
ELITISM_FACTOR = 2.0  # Extra reinforcement for global best
epsilon = 1e-6   # Small constant to prevent division by zero

# Initialize the unified pheromone matrix over dimensions:
# (exam, timeslot, room, examiner)
pheromone = np.ones((num_exams, num_timeslots, num_rooms, num_examiners)) * PHEROMONE_INIT

# Preinitialize a random "base" examiner heuristic for each (exam, timeslot, room, examiner)
base_examiner_heuristic = np.random.rand(num_exams, num_timeslots, num_rooms, num_examiners)

# ---------------------------
# Heuristic Calculation Function
# ---------------------------
def calculate_heuristic_matrix():
    """
    Compute a heuristic matrix for each exam, timeslot, room, and examiner.
    Combines:
      - Professor availability (binary factor)
      - Preferred timeslot bonus (1.0 if preferred, else 0.5)
      - Room capacity (favors larger rooms)
      - A base examiner heuristic (random factor)
      - A penalty from exam conflict counts (precomputed)
    """
    heuristic = np.zeros((num_exams, num_timeslots, num_rooms, num_examiners))
    for exam in range(num_exams):
        # Get the number of students registered for this exam.
        num_students_in_exam = np.sum(STUDENT_EXAM_MATRIX[:, exam])
        # For room capacity, we can optionally adjust if capacity is insufficient.
        for t in range(num_timeslots):
            # Professor availability factor (0 if unavailable, 1 if available)
            prof_avail = PROFESSOR_AVAILABILITY[exam, t]
            # Preferred timeslot bonus
            pref_bonus = 1.0 if t in PREFERRED_TIMES.get(exam, []) else 0.5
            for r in range(num_rooms):
                room_cap = ROOM_CAPACITY[r]
                # If room capacity is less than number of students, apply a penalty factor.
                room_factor = room_cap if num_students_in_exam <= room_cap else room_cap * 0.5
                for e in range(num_examiners):
                    # Combine factors; conflict_penalties[exam] is added in the denominator.
                    heuristic[exam, t, r, e] = (prof_avail * pref_bonus * room_factor * base_examiner_heuristic[exam, t, r, e]) / (1 + conflict_penalties[exam])
    return heuristic

# ---------------------------
# Evaluation Function
# ---------------------------
def evaluate_solution(solution):
    """
    Evaluate a candidate timetable solution.
    The solution is a dict mapping exam index to (timeslot, room, examiner).
    Penalties include:
      - Student exam overlaps (a student taking multiple exams in the same timeslot)
      - Professor unavailability (if professor is not available in the chosen timeslot)
      - Non-preferred timeslot (if chosen timeslot is not in preferred times)
      - Room capacity overload (if assigned room cannot hold all exam students)
      - Exam hall and examiner conflicts (if same hall or examiner is assigned more than once in a timeslot)
    Lower penalty indicates a better solution.
    """
    penalty = 0

    # 1. Student exam overlap penalty
    for student in range(num_students):
        exam_indices = np.where(STUDENT_EXAM_MATRIX[student] == 1)[0]
        # Gather the timeslots assigned to the exams this student takes.
        assigned_times = [solution[exam][0] for exam in exam_indices if exam in solution]
        # Count duplicate timeslot assignments.
        penalty += (len(assigned_times) - len(set(assigned_times))) * 5

    # 2. Per-exam penalties (professor availability, preferred times, room capacity)
    for exam, (t, r, e) in solution.items():
        # Professor availability check
        if not PROFESSOR_AVAILABILITY[exam, t]:
            penalty += 10
        # Preferred timeslot bonus check
        if t not in PREFERRED_TIMES.get(exam, []):
            penalty += 2
        # Room capacity check: if the number of students in exam exceeds room capacity, add penalty.
        num_students_in_exam = np.sum(STUDENT_EXAM_MATRIX[:, exam])
        if num_students_in_exam > ROOM_CAPACITY[r]:
            penalty += (num_students_in_exam - ROOM_CAPACITY[r]) * 5

    # 3. Conflict checks for exam halls and examiners per timeslot.
    # For each timeslot, check if an exam hall or examiner is assigned to more than one exam.
    for t in range(num_timeslots):
        hall_usage = {}
        examiner_usage = {}
        for exam, (ts, r, e) in solution.items():
            if ts == t:
                hall_usage[r] = hall_usage.get(r, 0) + 1
                examiner_usage[e] = examiner_usage.get(e, 0) + 1
        # If any hall or examiner is overused, add penalty.
        for usage in hall_usage.values():
            if usage > 1:
                penalty += (usage - 1) * 10
        for usage in examiner_usage.values():
            if usage > 1:
                penalty += (usage - 1) * 10

    return penalty

# ---------------------------
# Advanced ACO Algorithm
# ---------------------------
def advanced_aco_timetabling():
    global pheromone
    best_solution = None
    best_cost = float("inf")

    for iteration in range(NUM_ITERATIONS):
        solutions = []
        costs = []

        # Compute the heuristic matrix once per iteration.
        heuristic = calculate_heuristic_matrix()

        # Each ant constructs a solution.
        for ant in range(NUM_ANTS):
            solution = {}
            for exam in range(num_exams):
                # Build probability distribution for all (timeslot, room, examiner) combinations.
                prob_matrix = np.zeros((num_timeslots, num_rooms, num_examiners))
                for t in range(num_timeslots):
                    for r in range(num_rooms):
                        for e in range(num_examiners):
                            prob_matrix[t, r, e] = (pheromone[exam, t, r, e] ** ALPHA) * (heuristic[exam, t, r, e] ** BETA)

                # Normalize the probabilities (safeguard for zero sum).
                total = np.sum(prob_matrix)
                if total == 0:
                    probabilities = np.full(prob_matrix.shape, 1/(num_timeslots*num_rooms*num_examiners))
                else:
                    probabilities = prob_matrix / total

                # Choose one (timeslot, room, examiner) based on the computed probabilities.
                flat_index = np.random.choice(np.arange(num_timeslots * num_rooms * num_examiners), p=probabilities.flatten())
                chosen_t, chosen_r, chosen_e = np.unravel_index(flat_index, (num_timeslots, num_rooms, num_examiners))
                solution[exam] = (chosen_t, chosen_r, chosen_e)

            cost = evaluate_solution(solution)
            solutions.append(solution)
            costs.append(cost)

        # Pheromone evaporation.
        pheromone *= (1 - EVAPORATION_RATE)

        # Deposit pheromone based on each solution's quality.
        for sol, cost in zip(solutions, costs):
            for exam, (t, r, e) in sol.items():
                pheromone[exam, t, r, e] += 1.0 / (cost + epsilon)

        # Track the best solution found in this iteration.
        current_best_cost = min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_solution = solutions[costs.index(current_best_cost)]

        # Elitist update: reinforce the global best solution.
        for exam, (t, r, e) in best_solution.items():
            pheromone[exam, t, r, e] += ELITISM_FACTOR / (best_cost + epsilon)

        print(f"Iteration {iteration+1}/{NUM_ITERATIONS}: Best Cost = {best_cost}")

    return best_solution, best_cost

# ---------------------------
# Run the Advanced ACO Algorithm
# ---------------------------
best_timetable, best_cost = advanced_aco_timetabling()

# ---------------------------
# Display the Result
# ---------------------------
def display_timetable(timetable):
    print("\nExam\t--Timeslot--\t\t--Exam Hall--\t--Examiner--")
    for exam in range(num_exams):
        t, r, e = timetable[exam]
        print(f"{exams[exam]}\t{timeslot_labels[t]}\t{exam_halls[r]}\t\t{examiners[e]}")
    print("\nOverall Cost:", best_cost)

display_timetable(best_timetable)


Iteration 1/100: Best Cost = 2059
Iteration 2/100: Best Cost = 2059
Iteration 3/100: Best Cost = 1979
Iteration 4/100: Best Cost = 1979
Iteration 5/100: Best Cost = 1979
Iteration 6/100: Best Cost = 1979
Iteration 7/100: Best Cost = 1899
Iteration 8/100: Best Cost = 1899
Iteration 9/100: Best Cost = 1899
Iteration 10/100: Best Cost = 1899
Iteration 11/100: Best Cost = 1899
Iteration 12/100: Best Cost = 1899
Iteration 13/100: Best Cost = 1899
Iteration 14/100: Best Cost = 1899
Iteration 15/100: Best Cost = 1899
Iteration 16/100: Best Cost = 1899
Iteration 17/100: Best Cost = 1899
Iteration 18/100: Best Cost = 1889
Iteration 19/100: Best Cost = 1889
Iteration 20/100: Best Cost = 1884
Iteration 21/100: Best Cost = 1879
Iteration 22/100: Best Cost = 1839
Iteration 23/100: Best Cost = 1839
Iteration 24/100: Best Cost = 1839
Iteration 25/100: Best Cost = 1839
Iteration 26/100: Best Cost = 1839
Iteration 27/100: Best Cost = 1839
Iteration 28/100: Best Cost = 1839
Iteration 29/100: Best Cost =

In [5]:
# Advanced ACO for Integrated Exam Timetabling and Scheduling
# Google Colab–ready cell

import numpy as np
import random

# ---------------------------
# Reproducibility & Setup
# ---------------------------
np.random.seed(42)
random.seed(42)

# ---------------------------
# Problem Data
# ---------------------------
# Exams, timeslots, exam halls, and examiners
exams = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
timeslot_labels = ['9:00 AM to 11:00 AM', '12:00 PM to 2:00 PM']
exam_halls = ['Exam Hall 1', 'Exam Hall 2', 'Exam Hall 3']
examiners = ['Examiner 1', 'Examiner 2', 'Examiner 3', 'Examiner 4', 'Examiner 5']

num_exams = len(exams)
num_timeslots = len(timeslot_labels)
num_rooms = len(exam_halls)
num_examiners = len(examiners)
num_students = 100

# Room capacities corresponding to exam halls
ROOM_CAPACITY = [30, 50, 40]

# ---------------------------
# Input Data Generation
# ---------------------------
# Random binary matrix for students enrolled in exams (rows: students, columns: exams)
STUDENT_EXAM_MATRIX = np.random.randint(0, 2, (num_students, num_exams))

# Each exam is supervised by a professor; binary matrix for professor availability (exams x timeslots)
PROFESSOR_AVAILABILITY = np.random.randint(0, 2, (num_exams, num_timeslots))

# Define preferred times for some exams (if a professor has a preference, assign bonus)
PREFERRED_TIMES = {
    0: [0, 1],
    1: [2],
    2: [1, 3],   # even though there are only 2 timeslots here, we include extra values for illustration
    # You can extend this dictionary for more exams
}

# (Optional) Room adjacency can be used for further refinements
ROOM_ADJACENCY = np.random.rand(num_rooms, num_rooms)

# Precompute a conflict matrix:
# conflict_matrix[i,j] counts the number of students taking both exam i and exam j.
conflict_matrix = np.dot(STUDENT_EXAM_MATRIX.T, STUDENT_EXAM_MATRIX)
np.fill_diagonal(conflict_matrix, 0)
# For each exam, the total conflict penalty is the sum of conflicts with all other exams.
conflict_penalties = np.sum(conflict_matrix, axis=1)

# ---------------------------
# ACO Parameters
# ---------------------------
NUM_ANTS = 10
NUM_ITERATIONS = 100
PHEROMONE_INIT = 1.0
EVAPORATION_RATE = 0.5
ALPHA = 1.0    # Pheromone influence
BETA = 2.0     # Heuristic influence
ELITISM_FACTOR = 2.0  # Extra reinforcement for the global best solution
epsilon = 1e-6         # Small constant to avoid division by zero

# Initialize the pheromone matrix with dimensions: (exam, timeslot, room, examiner)
pheromone = np.ones((num_exams, num_timeslots, num_rooms, num_examiners)) * PHEROMONE_INIT

# ---------------------------
# Heuristic Calculation Function
# ---------------------------
def calculate_heuristic_matrix():
    """
    Compute a heuristic value for each combination (exam, timeslot, room, examiner).
    It combines:
      - Professor availability (0 or 1)
      - Preferred timeslot bonus (1.0 if preferred; else 0.5)
      - Room capacity (higher capacity favored, with penalty if insufficient)
      - A random examiner factor (to diversify assignments)
      - Conflict penalty from precomputed conflict_penalties
    """
    heuristic = np.zeros((num_exams, num_timeslots, num_rooms, num_examiners))
    # A base random factor for examiner assignment (adds diversity)
    base_examiner_factor = np.random.rand(num_exams, num_timeslots, num_rooms, num_examiners)

    for exam in range(num_exams):
        # Count number of students taking this exam
        num_students_in_exam = np.sum(STUDENT_EXAM_MATRIX[:, exam])
        for t in range(num_timeslots):
            # Professor availability factor
            prof_avail = PROFESSOR_AVAILABILITY[exam, t]
            # Preferred timeslot bonus
            pref_bonus = 1.0 if t in PREFERRED_TIMES.get(exam, []) else 0.5
            for r in range(num_rooms):
                room_cap = ROOM_CAPACITY[r]
                # Apply a penalty factor if room capacity is less than the number of students
                room_factor = room_cap if num_students_in_exam <= room_cap else room_cap * 0.5
                for e in range(num_examiners):
                    # Combine factors; higher conflict_penalties reduce the heuristic value.
                    heuristic[exam, t, r, e] = (prof_avail * pref_bonus * room_factor * base_examiner_factor[exam, t, r, e]) / (1 + conflict_penalties[exam])
    return heuristic

# ---------------------------
# Evaluation Function
# ---------------------------
def evaluate_solution(solution):
    """
    Evaluate a candidate timetable solution.
    The solution is a dictionary mapping exam index to a tuple (timeslot, room, examiner).
    Penalties include:
      1. Student exam overlaps: if a student is scheduled for multiple exams in the same timeslot.
      2. Professor unavailability: heavy penalty if a professor is not available.
      3. Non-preferred timeslot: penalty if a timeslot is not in the preferred list.
      4. Room capacity overload: penalty proportional to the overflow.
      5. Duplicate usage of exam halls or examiners in the same timeslot.
    Lower penalty indicates a better solution.
    """
    penalty = 0
    # 1. Student overlap penalty
    for student in range(num_students):
        exams_taken = np.where(STUDENT_EXAM_MATRIX[student] == 1)[0]
        assigned_times = [solution[exam][0] for exam in exams_taken if exam in solution]
        penalty += (len(assigned_times) - len(set(assigned_times))) * 5

    # 2. Per-exam penalties
    for exam, (t, r, e) in solution.items():
        # Professor availability check
        if not PROFESSOR_AVAILABILITY[exam, t]:
            penalty += 10
        # Non-preferred timeslot penalty
        if t not in PREFERRED_TIMES.get(exam, []):
            penalty += 2
        # Room capacity check
        num_students_in_exam = np.sum(STUDENT_EXAM_MATRIX[:, exam])
        if num_students_in_exam > ROOM_CAPACITY[r]:
            penalty += (num_students_in_exam - ROOM_CAPACITY[r]) * 5

    # 3. Conflict in exam hall and examiner usage per timeslot
    for t in range(num_timeslots):
        hall_usage = {}
        examiner_usage = {}
        for exam, (ts, r, e) in solution.items():
            if ts == t:
                hall_usage[r] = hall_usage.get(r, 0) + 1
                examiner_usage[e] = examiner_usage.get(e, 0) + 1
        for usage in hall_usage.values():
            if usage > 1:
                penalty += (usage - 1) * 10
        for usage in examiner_usage.values():
            if usage > 1:
                penalty += (usage - 1) * 10

    return penalty

# ---------------------------
# Advanced ACO Algorithm
# ---------------------------
def advanced_aco_timetabling():
    global pheromone
    best_solution = None
    best_cost = float("inf")

    for iteration in range(NUM_ITERATIONS):
        solutions = []
        costs = []
        # Compute heuristic matrix once per iteration
        heuristic = calculate_heuristic_matrix()

        # Each ant constructs a solution for all exams
        for ant in range(NUM_ANTS):
            solution = {}
            for exam in range(num_exams):
                # Build a probability distribution for all (timeslot, room, examiner) combinations
                prob_matrix = np.zeros((num_timeslots, num_rooms, num_examiners))
                for t in range(num_timeslots):
                    for r in range(num_rooms):
                        for e in range(num_examiners):
                            prob_matrix[t, r, e] = (pheromone[exam, t, r, e] ** ALPHA) * (heuristic[exam, t, r, e] ** BETA)
                # Normalize the probability distribution; if zero, choose uniformly
                total = np.sum(prob_matrix)
                if total == 0:
                    probabilities = np.full(prob_matrix.shape, 1 / (num_timeslots * num_rooms * num_examiners))
                else:
                    probabilities = prob_matrix / total

                # Sample one assignment (flatten the probability matrix for sampling)
                flat_index = np.random.choice(np.arange(num_timeslots * num_rooms * num_examiners), p=probabilities.flatten())
                chosen_t, chosen_r, chosen_e = np.unravel_index(flat_index, (num_timeslots, num_rooms, num_examiners))
                solution[exam] = (chosen_t, chosen_r, chosen_e)

            cost = evaluate_solution(solution)
            solutions.append(solution)
            costs.append(cost)

        # Pheromone evaporation step
        pheromone *= (1 - EVAPORATION_RATE)
        # Pheromone deposit for each solution
        for sol, cost in zip(solutions, costs):
            for exam, (t, r, e) in sol.items():
                pheromone[exam, t, r, e] += 1.0 / (cost + epsilon)

        # Track the best solution of the iteration
        current_best_cost = min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_solution = solutions[costs.index(current_best_cost)]

        # Elitist update: reinforce the global best solution extra
        for exam, (t, r, e) in best_solution.items():
            pheromone[exam, t, r, e] += ELITISM_FACTOR / (best_cost + epsilon)

        print(f"Iteration {iteration + 1}/{NUM_ITERATIONS}: Best Cost = {best_cost}")

    return best_solution, best_cost

# ---------------------------
# Run the Advanced ACO Algorithm
# ---------------------------
best_timetable, best_cost = advanced_aco_timetabling()

# ---------------------------
# Display the Final Timetable
# ---------------------------
def display_timetable(timetable):
    print("\nExam\t--Timeslot--\t\t--Exam Hall--\t--Examiner--")
    for exam in range(num_exams):
        t, r, e = timetable[exam]
        print(f"{exams[exam]}\t{timeslot_labels[t]}\t{exam_halls[r]}\t{examiners[e]}")
    print("\nOverall Cost:", best_cost)

display_timetable(best_timetable)


Iteration 1/100: Best Cost = 2041
Iteration 2/100: Best Cost = 2041
Iteration 3/100: Best Cost = 2041
Iteration 4/100: Best Cost = 2003
Iteration 5/100: Best Cost = 2001
Iteration 6/100: Best Cost = 2001
Iteration 7/100: Best Cost = 1993
Iteration 8/100: Best Cost = 1891
Iteration 9/100: Best Cost = 1891
Iteration 10/100: Best Cost = 1891
Iteration 11/100: Best Cost = 1891
Iteration 12/100: Best Cost = 1891
Iteration 13/100: Best Cost = 1891
Iteration 14/100: Best Cost = 1891
Iteration 15/100: Best Cost = 1891
Iteration 16/100: Best Cost = 1891
Iteration 17/100: Best Cost = 1891
Iteration 18/100: Best Cost = 1891
Iteration 19/100: Best Cost = 1891
Iteration 20/100: Best Cost = 1891
Iteration 21/100: Best Cost = 1891
Iteration 22/100: Best Cost = 1891
Iteration 23/100: Best Cost = 1891
Iteration 24/100: Best Cost = 1891
Iteration 25/100: Best Cost = 1861
Iteration 26/100: Best Cost = 1861
Iteration 27/100: Best Cost = 1861
Iteration 28/100: Best Cost = 1861
Iteration 29/100: Best Cost =

In [6]:
# Advanced ACO for Integrated Exam Timetabling and Scheduling with Accuracy Checking
# Google Colab–ready cell

import numpy as np
import random

# ---------------------------
# Reproducibility & Setup
# ---------------------------
np.random.seed(42)
random.seed(42)

# ---------------------------
# Problem Data
# ---------------------------
exams = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
timeslot_labels = ['9:00 AM to 11:00 AM', '12:00 PM to 2:00 PM']
exam_halls = ['Exam Hall 1', 'Exam Hall 2', 'Exam Hall 3']
examiners = ['Examiner 1', 'Examiner 2', 'Examiner 3', 'Examiner 4', 'Examiner 5']

num_exams = len(exams)
num_timeslots = len(timeslot_labels)
num_rooms = len(exam_halls)
num_examiners = len(examiners)
num_students = 100

# Room capacities corresponding to exam halls
ROOM_CAPACITY = [30, 50, 40]

# ---------------------------
# Input Data Generation
# ---------------------------
STUDENT_EXAM_MATRIX = np.random.randint(0, 2, (num_students, num_exams))
PROFESSOR_AVAILABILITY = np.random.randint(0, 2, (num_exams, num_timeslots))
PREFERRED_TIMES = {0: [0, 1], 1: [0], 2: [1]}  # Example preferences
ROOM_ADJACENCY = np.random.rand(num_rooms, num_rooms)

# Precompute conflict matrix & penalties:
conflict_matrix = np.dot(STUDENT_EXAM_MATRIX.T, STUDENT_EXAM_MATRIX)
np.fill_diagonal(conflict_matrix, 0)
conflict_penalties = np.sum(conflict_matrix, axis=1)

# ---------------------------
# ACO Parameters
# ---------------------------
NUM_ANTS = 10
NUM_ITERATIONS = 100
PHEROMONE_INIT = 1.0
EVAPORATION_RATE = 0.5
ALPHA = 1.0
BETA = 2.0
ELITISM_FACTOR = 2.0
epsilon = 1e-6

# Initialize pheromone matrix: dimensions (exam, timeslot, room, examiner)
pheromone = np.ones((num_exams, num_timeslots, num_rooms, num_examiners)) * PHEROMONE_INIT

# ---------------------------
# Heuristic Calculation Function
# ---------------------------
def calculate_heuristic_matrix():
    heuristic = np.zeros((num_exams, num_timeslots, num_rooms, num_examiners))
    # Introduce a random factor to diversify examiner assignments.
    base_examiner_factor = np.random.rand(num_exams, num_timeslots, num_rooms, num_examiners)
    for exam in range(num_exams):
        num_students_in_exam = np.sum(STUDENT_EXAM_MATRIX[:, exam])
        for t in range(num_timeslots):
            prof_avail = PROFESSOR_AVAILABILITY[exam, t]
            pref_bonus = 1.0 if t in PREFERRED_TIMES.get(exam, []) else 0.5
            for r in range(num_rooms):
                room_cap = ROOM_CAPACITY[r]
                room_factor = room_cap if num_students_in_exam <= room_cap else room_cap * 0.5
                for e in range(num_examiners):
                    heuristic[exam, t, r, e] = (prof_avail * pref_bonus * room_factor * base_examiner_factor[exam, t, r, e]) / (1 + conflict_penalties[exam])
    return heuristic

# ---------------------------
# Evaluation Function
# ---------------------------
def evaluate_solution(solution):
    penalty = 0
    # Student overlap penalty: multiple exams in same timeslot for a student.
    for student in range(num_students):
        exams_taken = np.where(STUDENT_EXAM_MATRIX[student] == 1)[0]
        assigned_times = [solution[exam][0] for exam in exams_taken if exam in solution]
        penalty += (len(assigned_times) - len(set(assigned_times))) * 5
    # Per-exam penalties.
    for exam, (t, r, e) in solution.items():
        if not PROFESSOR_AVAILABILITY[exam, t]:
            penalty += 10
        if t not in PREFERRED_TIMES.get(exam, []):
            penalty += 2
        num_students_in_exam = np.sum(STUDENT_EXAM_MATRIX[:, exam])
        if num_students_in_exam > ROOM_CAPACITY[r]:
            penalty += (num_students_in_exam - ROOM_CAPACITY[r]) * 5
    # Duplicate assignment check per timeslot.
    for t in range(num_timeslots):
        hall_usage = {}
        examiner_usage = {}
        for exam, (ts, r, e) in solution.items():
            if ts == t:
                hall_usage[r] = hall_usage.get(r, 0) + 1
                examiner_usage[e] = examiner_usage.get(e, 0) + 1
        for usage in hall_usage.values():
            if usage > 1:
                penalty += (usage - 1) * 10
        for usage in examiner_usage.values():
            if usage > 1:
                penalty += (usage - 1) * 10
    return penalty

# ---------------------------
# Advanced ACO Algorithm
# ---------------------------
def advanced_aco_timetabling():
    global pheromone
    best_solution = None
    best_cost = float("inf")

    for iteration in range(NUM_ITERATIONS):
        solutions = []
        costs = []
        heuristic = calculate_heuristic_matrix()

        for ant in range(NUM_ANTS):
            solution = {}
            for exam in range(num_exams):
                prob_matrix = np.zeros((num_timeslots, num_rooms, num_examiners))
                for t in range(num_timeslots):
                    for r in range(num_rooms):
                        for e in range(num_examiners):
                            prob_matrix[t, r, e] = (pheromone[exam, t, r, e] ** ALPHA) * (heuristic[exam, t, r, e] ** BETA)
                total = np.sum(prob_matrix)
                if total == 0:
                    probabilities = np.full(prob_matrix.shape, 1 / (num_timeslots * num_rooms * num_examiners))
                else:
                    probabilities = prob_matrix / total
                flat_index = np.random.choice(np.arange(num_timeslots * num_rooms * num_examiners), p=probabilities.flatten())
                chosen_t, chosen_r, chosen_e = np.unravel_index(flat_index, (num_timeslots, num_rooms, num_examiners))
                solution[exam] = (chosen_t, chosen_r, chosen_e)
            cost = evaluate_solution(solution)
            solutions.append(solution)
            costs.append(cost)

        pheromone *= (1 - EVAPORATION_RATE)
        for sol, cost in zip(solutions, costs):
            for exam, (t, r, e) in sol.items():
                pheromone[exam, t, r, e] += 1.0 / (cost + epsilon)

        current_best_cost = min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_solution = solutions[costs.index(current_best_cost)]

        for exam, (t, r, e) in best_solution.items():
            pheromone[exam, t, r, e] += ELITISM_FACTOR / (best_cost + epsilon)

        print(f"Iteration {iteration + 1}/{NUM_ITERATIONS}: Best Cost = {best_cost}")

    return best_solution, best_cost

# ---------------------------
# Accuracy Checking Function
# ---------------------------
def check_solution_accuracy(solution):
    """
    Check the solution against key constraints.
    Returns a list of violation messages. An empty list indicates no violations.
    """
    violations = []
    for exam, (t, r, e) in solution.items():
        if not PROFESSOR_AVAILABILITY[exam, t]:
            violations.append(f"Exam {exams[exam]} scheduled in timeslot {t} when professor is unavailable.")
        if t not in PREFERRED_TIMES.get(exam, []):
            violations.append(f"Exam {exams[exam]} scheduled in non-preferred timeslot {t}.")
        num_students_in_exam = np.sum(STUDENT_EXAM_MATRIX[:, exam])
        if num_students_in_exam > ROOM_CAPACITY[r]:
            violations.append(f"Exam {exams[exam]} assigned to room {exam_halls[r]} with capacity {ROOM_CAPACITY[r]} but has {num_students_in_exam} students.")
    for t in range(num_timeslots):
        hall_usage = {}
        examiner_usage = {}
        for exam, (ts, r, e) in solution.items():
            if ts == t:
                hall_usage[r] = hall_usage.get(r, 0) + 1
                examiner_usage[e] = examiner_usage.get(e, 0) + 1
        for r, usage in hall_usage.items():
            if usage > 1:
                violations.append(f"Timeslot {t}: Exam Hall {exam_halls[r]} used {usage} times.")
        for e, usage in examiner_usage.items():
            if usage > 1:
                violations.append(f"Timeslot {t}: Examiner {examiners[e]} assigned {usage} times.")
    return violations

# ---------------------------
# Test Accuracy over Multiple Runs
# ---------------------------
def test_aco_accuracy(num_runs=5):
    best_costs = []
    for i in range(num_runs):
        print(f"\n=== Run {i+1} ===")
        sol, cost = advanced_aco_timetabling()
        best_costs.append(cost)
        print(f"Final Best Cost: {cost}")
        violations = check_solution_accuracy(sol)
        if violations:
            print("Violations Found:")
            for v in violations:
                print("  -", v)
        else:
            print("No violations found. The solution meets all constraints.")
    print("\nAverage Best Cost over", num_runs, "runs:", np.mean(best_costs))

# ---------------------------
# Run the ACO and Check Accuracy
# ---------------------------
best_timetable, best_cost = advanced_aco_timetabling()
print("\n=== Final Best Timetable ===")
def display_timetable(timetable):
    print("\nExam\t--Timeslot--\t\t--Exam Hall--\t--Examiner--")
    for exam in range(num_exams):
        t, r, e = timetable[exam]
        print(f"{exams[exam]}\t{timeslot_labels[t]}\t{exam_halls[r]}\t{examiners[e]}")
    print("\nOverall Cost:", best_cost)

display_timetable(best_timetable)

# Check accuracy of the obtained solution
violations = check_solution_accuracy(best_timetable)
if violations:
    print("\nSolution Violations:")
    for v in violations:
        print("  -", v)
else:
    print("\nThe final solution meets all constraints.")

# Optionally, run multiple tests to assess average performance and constraint satisfaction.
# Uncomment the next line to run the test accuracy function:
# test_aco_accuracy(num_runs=5)


Iteration 1/100: Best Cost = 2041
Iteration 2/100: Best Cost = 2041
Iteration 3/100: Best Cost = 2041
Iteration 4/100: Best Cost = 2001
Iteration 5/100: Best Cost = 2001
Iteration 6/100: Best Cost = 2001
Iteration 7/100: Best Cost = 1991
Iteration 8/100: Best Cost = 1889
Iteration 9/100: Best Cost = 1889
Iteration 10/100: Best Cost = 1889
Iteration 11/100: Best Cost = 1889
Iteration 12/100: Best Cost = 1889
Iteration 13/100: Best Cost = 1889
Iteration 14/100: Best Cost = 1889
Iteration 15/100: Best Cost = 1889
Iteration 16/100: Best Cost = 1889
Iteration 17/100: Best Cost = 1889
Iteration 18/100: Best Cost = 1889
Iteration 19/100: Best Cost = 1889
Iteration 20/100: Best Cost = 1889
Iteration 21/100: Best Cost = 1889
Iteration 22/100: Best Cost = 1889
Iteration 23/100: Best Cost = 1889
Iteration 24/100: Best Cost = 1889
Iteration 25/100: Best Cost = 1851
Iteration 26/100: Best Cost = 1851
Iteration 27/100: Best Cost = 1851
Iteration 28/100: Best Cost = 1851
Iteration 29/100: Best Cost =